In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import mie_params_of_vle_visc as mie_params_of


PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
#######################################
# type and optimization configuration #
#######################################

of_type = "vle_visc"
# ranges for sigma, eps, lambda_r
l_bounds = [2.5, 100., 9.]
u_bounds = [3.5, 300., 26.]
m_base = 5
params_file = f'optimized_mie_params_{of_type}.csv'

In [3]:
filename = '../../computed_files/phase_equilibria_fluid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp}

In [4]:
######################
# Loading FE-ANN EoS #
######################

ckpt_folder = '../../ann_models/feann_eos'

prefix_params = 'FE-ANN-EoS-params_'

###
Tscale = 'Tinv'
seed = 1
factor = 0.05
EPOCHS = 20000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
fun_dic_fluid = helper_jitted_funs(helmholtz_model, helmholtz_params)

#########################
# Shear visocsity model #
#########################

activation_dicts = {'linear': linear_activation, 'softplus': nn.softplus}
folder_visc = '../../ann_models/visc_models'
hidden_layers = 2
neurons = 30
prefix = 'logvisc-rho-Tinv-penalty'
ckpt_dir = folder_visc
seed = 0
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logvisc = {'params': state_restored['params']}
logvisc_model = TransportModel_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

logvisc_model_jit = jax.jit(lambda alpha, rhoad, Tad: logvisc_model.apply(params_logvisc, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
visc_fun = lambda alpha, rhoad, Tad: jnp.exp(logvisc_model_jit(alpha, rhoad, Tad))

In [5]:
# Reading data from NIST
filename = 'cf4.xlsx'
DataFile = pd.ExcelFile(filename)

if not os.path.exists(params_file):
    sampler = qmc.Sobol(d=3, scramble=False)
    sample = sampler.random_base2(m=m_base)

    scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

    sigma_guess = scaled_sample[:, 0]
    epsilon_guess = scaled_sample[:, 1]
    lr_guess = scaled_sample[:, 2]

    n = len(sigma_guess)

    index = np.arange(n)
    solved = np.zeros(n, dtype=bool)
    sigma_sol = np.nan * np.ones(n)
    epsilon_sol = np.nan * np.ones(n)
    lr_sol = np.nan * np.ones(n)
    of = np.nan * np.ones(n)
    solved_success = np.zeros(n, dtype=bool)

    df = pd.DataFrame({'index': index, 
                    'sigma_guess': sigma_guess, 'epsilon_guess': epsilon_guess, 'lr_guess': lr_guess,
                    'optimized': solved, 
                    'sigma_sol': sigma_sol, 'epsilon_sol': epsilon_sol, 'lr_sol': lr_sol, 'of': of, 'solved_success': solved_success})
    
    df.to_csv(params_file, index=False)




In [6]:
# bounds to control sigma, epsilon and lr
bounds = ((None, None), (None, None), (7., 34.))

kwargs = {'DataFile': DataFile,
          'fun_dic': fun_dic_fluid,
          'visc_fun': visc_fun,
          'interpd_dict': interpd_dict,
          'lambda_a': 6.,
          'weight_rhov_vle':0.0,
          'weight_hvap':1.0,
          'add_critical_point': True,
          'weight_visc': 1.0,   
          }
args = tuple(kwargs.values())

In [7]:
df = pd.read_csv(params_file)
n = len(df)
for i in range(n):
    if not df.loc[i, 'optimized']:
        inc0 = df.loc[i, ['sigma_guess', 'epsilon_guess', 'lr_guess']].to_list()
        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds, options={'maxiter':10})

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)
    print(i, df.loc[i, 'optimized'], df.loc[i, 'of'])

0 True 3.8991321204032823
1 True 0.0859570501621682
2 True 0.7683804535657979
3 True 0.5708250289436924
4 True 1.6893812245452398
5 True 0.266569646681022
6 True 3.485668016939228
7 True 0.085957050161974
8 True 1.639263129193345
9 True 0.5431673106949407
10 True 3.4699530316818072
11 True 0.3347554990398123
12 True 1.0358975087214333
13 True 0.1203185694090945
14 True 0.1997809324134971
15 True 7.704851833249274
16 True 2.9185424266735707
17 True 1.209973115459737
18 True 0.5171292267849648
19 True 0.2114823151985556
20 True 3.4992652062694587
21 True 0.0859570501621994
22 True 0.3229860118576323
23 True 2.515003915290712
24 True 4.137633357517507
25 True 0.0859570501621037
26 True 0.8722398934389735
27 True 1.138560729204282
28 True 1.6706757760887174
29 True 0.1863337856437519
30 True 3.471030481757169
31 True 6.498650891758102


In [8]:
df = pd.read_csv(params_file)
n = len(df)

of_order = np.argsort(df['of'].to_numpy())

for i in of_order[:5]:
    if not df.loc[i, 'solved_success']:
        inc0 = df.loc[i, ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()

        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds)

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)


In [9]:
of_order = np.argsort(df['of'].to_numpy())
df.loc[of_order].head(n=10)

,index,sigma_guess,epsilon_guess,lr_guess,optimized,sigma_sol,epsilon_sol,lr_sol,of,solved_success
7,7,2.62500,225.00,15.37500,True,4.399893,191.171195,12.589385,0.085957,True
25,25,3.15625,231.25,9.53125,True,4.399893,191.171194,12.589384,0.085957,True
1,1,3.00000,200.00,17.50000,True,4.399893,191.171243,12.589391,0.085957,True
21,21,3.46875,218.75,14.84375,True,4.399894,191.171274,12.589397,0.085957,True
13,13,3.31250,237.50,22.81250,True,4.599070,233.328430,22.062385,0.120319,True
29,29,3.28125,256.25,20.15625,True,4.576871,236.530350,21.325810,0.186334,False
14,14,3.06250,187.50,10.06250,True,4.258513,175.231481,9.849241,0.199781,False
19,19,2.84375,243.75,21.21875,True,4.370949,197.708333,12.338310,0.211482,False
5,5,3.37500,275.00,11.12500,True,4.050000,192.500000,11.959375,0.266570,False
22,22,3.21875,168.75,19.09375,True,4.041319,200.625000,13.789931,0.322986,False


In [10]:
param_best = df.loc[of_order[0], ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()
param_best

[4.3998934392818185, 191.17119538636152, 12.589385180890904]